In [ ]:
%%capture
pip install pandas 
pip install geopandas 
pip install requests 

In [19]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

In [20]:
shp_500 = gpd.read_file("/Volumes/Backup Plus/kursovaya_4/data/vyborka/nt/nt_buf500.gpkg", encoding='latin1')
shp_1500 = gpd.read_file("/Volumes/Backup Plus/kursovaya_4/data/vyborka/nt/nt_buf1500.gpkg", encoding='latin1')
shp_500.head()

,id,left,top,right,bottom,geometry
0,10177,666110.5061,6.421885e+06,666140.5061,6.421915e+06,"MULTIPOLYGON (((666610.506 6421885.199, 666608..."
1,10178,666110.5061,6.421855e+06,666140.5061,6.421885e+06,"MULTIPOLYGON (((666610.506 6421855.199, 666608..."
2,10975,666140.5061,6.422095e+06,666170.5061,6.422125e+06,"MULTIPOLYGON (((666640.506 6422095.199, 666638..."
3,10976,666140.5061,6.422065e+06,666170.5061,6.422095e+06,"MULTIPOLYGON (((666640.506 6422065.199, 666638..."
4,10977,666140.5061,6.422035e+06,666170.5061,6.422065e+06,"MULTIPOLYGON (((666640.506 6422035.199, 666638..."


In [21]:
shp_500.describe()

,id,left,top,right,bottom
count,337671.000000,337671.000000,3.376710e+05,337671.000000,3.376710e+05
mean,278241.605338,676104.470850,6.425089e+06,676134.470850,6.425119e+06
std,130881.453729,4876.734655,5.205587e+03,4876.734655,5.205587e+03
min,10177.000000,666110.506100,6.413545e+06,666140.506100,6.413575e+06
25%,172532.500000,672170.506100,6.421135e+06,672200.506100,6.421165e+06
50%,281049.000000,676220.506100,6.424975e+06,676250.506100,6.425005e+06
75%,383041.500000,680000.506100,6.429145e+06,680030.506100,6.429175e+06
max,572775.000000,687080.506100,6.436315e+06,687110.506100,6.436345e+06


In [22]:
shp_1500.describe()

,id,left,top,right,bottom
count,337671.000000,337671.000000,3.376710e+05,337671.000000,3.376710e+05
mean,278241.605338,676104.470850,6.425089e+06,676134.470850,6.425119e+06
std,130881.453729,4876.734655,5.205587e+03,4876.734655,5.205587e+03
min,10177.000000,666110.506100,6.413545e+06,666140.506100,6.413575e+06
25%,172532.500000,672170.506100,6.421135e+06,672200.506100,6.421165e+06
50%,281049.000000,676220.506100,6.424975e+06,676250.506100,6.425005e+06
75%,383041.500000,680000.506100,6.429145e+06,680030.506100,6.429175e+06
max,572775.000000,687080.506100,6.436315e+06,687110.506100,6.436345e+06


In [23]:
# extract the geometry in GeoJSON format
geoms_500 = shp_500.geometry.values # list of shapely geometries
geoms_1500 = shp_1500.geometry.values

In [24]:
# transform to GeJSON format
from shapely.geometry import mapping
geoms_500 = [mapping(elem) for elem in geoms_500 if elem is not None]
geoms_1500 = [mapping(elem) for elem in geoms_1500 if elem is not None]
#geoms = res_geom
res_geoms_500 = []
res_geoms_1500 = []

In [26]:
shp_500["heights_diff"] = np.nan
with rasterio.open("/Volumes/Backup Plus/kursovaya_4/data/vyborka/nt/nt_relief.tif") as src:
    for index, cur_geom in tqdm(enumerate(geoms_500)):
        out_image_500, out_transform = mask(src, [cur_geom], crop=True)
        out_image_500 = out_image_500[0]
        tmp_need_500 = out_image_500[np.where(~np.isclose(out_image_500, np.min(out_image_500)))]
        shp_500.at[index, 'dispersion'] = np.var(tmp_need_500)
        if len(tmp_need_500) > 0:
            shp_500.at[index, 'heights_diff'] = np.max(tmp_need_500) - np.min(tmp_need_500)
        else:
            shp_500.at[index, 'heights_diff'] = 0
        shp_500.at[index, 'median'] = np.median((tmp_need_500))
        shp_500.at[index, 'qntl0'] = np.quantile((tmp_need_500),0)
        shp_500.at[index, 'qntl25'] = np.quantile((tmp_need_500),.25, axis = 0)
        shp_500.at[index, 'qntl75'] = np.quantile((tmp_need_500),.75, axis = 0)
        shp_500.at[index, 'qntl1'] = np.quantile((tmp_need_500),1)
    print(shp_500)
    
    

337671it [13:28, 417.89it/s]

            id         left           top        right        bottom  \
0        10177  666110.5061  6.421885e+06  666140.5061  6.421915e+06   
1        10178  666110.5061  6.421855e+06  666140.5061  6.421885e+06   
2        10975  666140.5061  6.422095e+06  666170.5061  6.422125e+06   
3        10976  666140.5061  6.422065e+06  666170.5061  6.422095e+06   
4        10977  666140.5061  6.422035e+06  666170.5061  6.422065e+06   
...        ...          ...           ...          ...           ...   
337666  572771  687080.5061  6.424915e+06  687110.5061  6.424945e+06   
337667  572772  687080.5061  6.424885e+06  687110.5061  6.424915e+06   
337668  572773  687080.5061  6.424855e+06  687110.5061  6.424885e+06   
337669  572774  687080.5061  6.424825e+06  687110.5061  6.424855e+06   
337670  572775  687080.5061  6.424795e+06  687110.5061  6.424825e+06   

                                                 geometry  heights_diff  \
0       MULTIPOLYGON (((666610.506 6421885.199, 666608...   

In [27]:
shp_500.head()

,id,left,top,right,bottom,geometry,heights_diff,dispersion,median,qntl0,qntl25,qntl75,qntl1
0,10177,666110.5061,6.421885e+06,666140.5061,6.421915e+06,"MULTIPOLYGON (((666610.506 6421885.199, 666608...",55.890167,108.935768,254.781082,228.534271,247.125725,263.367111,284.424438
1,10178,666110.5061,6.421855e+06,666140.5061,6.421885e+06,"MULTIPOLYGON (((666610.506 6421855.199, 666608...",54.534637,105.067314,254.418213,228.534271,246.638763,263.055534,283.068909
2,10975,666140.5061,6.422095e+06,666170.5061,6.422125e+06,"MULTIPOLYGON (((666640.506 6422095.199, 666638...",55.890167,104.147598,258.662628,228.534271,249.392708,264.663574,284.424438
3,10976,666140.5061,6.422065e+06,666170.5061,6.422095e+06,"MULTIPOLYGON (((666640.506 6422065.199, 666638...",55.890167,106.714104,258.410980,228.534271,249.256195,264.678162,284.424438
4,10977,666140.5061,6.422035e+06,666170.5061,6.422065e+06,"MULTIPOLYGON (((666640.506 6422035.199, 666638...",55.890167,109.221397,258.071960,228.534271,248.913116,264.663574,284.424438


In [28]:
shp_1500["heights_diff"] = np.nan
with rasterio.open("/Volumes/Backup Plus/kursovaya_4/data/vyborka/nt/nt_relief.tif") as src:
    for index, cur_geom in tqdm(enumerate(geoms_1500)):
        out_image_1500, out_transform = mask(src, [cur_geom], crop=True)
        out_image_1500 = out_image_1500[0]
        tmp_need_1500 = out_image_1500[np.where(~np.isclose(out_image_1500, np.min(out_image_1500)))]
        shp_1500.at[index, 'dispersion'] = np.var(tmp_need_1500)
        if len(tmp_need_1500) > 0:
            shp_1500.at[index, 'heights_diff'] = np.max(tmp_need_1500) - np.min(tmp_need_1500)
        else:
            shp_1500.at[index, 'heights_diff'] = 0
        shp_1500.at[index, 'median'] = np.median((tmp_need_1500))
        shp_1500.at[index, 'qntl0'] = np.quantile((tmp_need_1500),0)
        shp_1500.at[index, 'qntl25'] = np.quantile((tmp_need_1500),.25, axis = 0)
        shp_1500.at[index, 'qntl75'] = np.quantile((tmp_need_1500),.75, axis = 0)
        shp_1500.at[index, 'qntl1'] = np.quantile((tmp_need_1500),1)
    print(shp_1500)

337671it [19:38, 286.60it/s]

            id         left           top        right        bottom  \
0        10177  666110.5061  6.421885e+06  666140.5061  6.421915e+06   
1        10178  666110.5061  6.421855e+06  666140.5061  6.421885e+06   
2        10975  666140.5061  6.422095e+06  666170.5061  6.422125e+06   
3        10976  666140.5061  6.422065e+06  666170.5061  6.422095e+06   
4        10977  666140.5061  6.422035e+06  666170.5061  6.422065e+06   
...        ...          ...           ...          ...           ...   
337666  572771  687080.5061  6.424915e+06  687110.5061  6.424945e+06   
337667  572772  687080.5061  6.424885e+06  687110.5061  6.424915e+06   
337668  572773  687080.5061  6.424855e+06  687110.5061  6.424885e+06   
337669  572774  687080.5061  6.424825e+06  687110.5061  6.424855e+06   
337670  572775  687080.5061  6.424795e+06  687110.5061  6.424825e+06   

                                                 geometry  heights_diff  \
0       MULTIPOLYGON (((667610.506 6421885.199, 667605...   

In [29]:
shp_1500.head()

,id,left,top,right,bottom,geometry,heights_diff,dispersion,median,qntl0,qntl25,qntl75,qntl1
0,10177,666110.5061,6.421885e+06,666140.5061,6.421915e+06,"MULTIPOLYGON (((667610.506 6421885.199, 667605...",102.727234,357.304016,255.830521,208.0,242.188278,265.930176,310.727234
1,10178,666110.5061,6.421855e+06,666140.5061,6.421885e+06,"MULTIPOLYGON (((667610.506 6421855.199, 667605...",106.793823,364.843567,256.172729,208.0,242.455643,266.183807,314.793823
2,10975,666140.5061,6.422095e+06,666170.5061,6.422125e+06,"MULTIPOLYGON (((667640.506 6422095.199, 667635...",96.162354,321.052246,254.087692,208.0,240.315861,264.246315,304.162354
3,10976,666140.5061,6.422065e+06,666170.5061,6.422095e+06,"MULTIPOLYGON (((667640.506 6422065.199, 667635...",92.771362,323.485291,254.351959,208.0,240.540710,264.545258,300.771362
4,10977,666140.5061,6.422035e+06,666170.5061,6.422065e+06,"MULTIPOLYGON (((667640.506 6422035.199, 667635...",91.251282,326.709717,254.629272,208.0,240.840233,264.788818,299.251282


In [30]:
shp_500.to_file("/Users/panssies/Documents/script4/nt_result500.shp")
shp_1500.to_file("/Users/panssies/Documents/script4/nt_result1500.shp")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  
